# Major programs

To assist in understanding what major programs should be prioritized for inclusion or removal in the Compliance Supplement, it would be nice to see major program counts by ALN.

In short:

1. For each year, look at every federal award made.
2. For each award, get the ALN, and add one.


In [1]:
%pip install openpyxl
%pip install requests
import json
import openpyxl
import os
import requests

from collections import defaultdict


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load our config
# It has three keys: 
#  * YEARS, an array of years (2023, 2022, etc.)
#  * FAC_API, the base URL for the analysis
#  * FAC_API_KEY, the API key.
config = json.load(open("config.json", "r"))

## fetch_all

A helper to fetch all of a result set.

Counts queries.

In [3]:
query_counts = defaultdict(int)

def fetch_all(base, endpoint, params):
    fetching = True
    results = []
    offset = 0
    inc = 20000
    while fetching:
        params = params | {
            "offset": offset,
            "limit": ((offset+inc)-1)
        }
        res = requests.get(f"{base}/{endpoint}",
                           params=params,
                           headers={
                               "x-api-key": config["FAC_API_KEY"]
                           }
                           )
        query_counts[endpoint] += 1
        resj = res.json()
        # print(f"{offset} {len(resj)}")
        if not res or "code" in resj or len(resj) == 0:
            fetching = False
            break
        else:
            results += resj
            offset += inc
    return results


In [4]:
# For each year, get every report.
# Then, get the unique set of ALNs associated with that report.
alns_by_year = defaultdict(lambda: defaultdict(set))
for year in config["YEARS"]:
    awards = fetch_all(config["FAC_API"], "federal_awards", {"audit_year": f"eq.{year}"})
    for award in awards:
        ALN = award["federal_agency_prefix"] + "." + award["federal_award_extension"]
        alns_by_year[year][ALN].add(award["report_id"])

## Do some counting

Now that we grabbed everything, we can do some counting.

We want to count the incidence of each ALN *per report per year*.

In [5]:
# We'll use a dictionary of dictionaries
yearly_aln_counts = defaultdict(lambda: defaultdict(int))

for year, ALNs in alns_by_year.items():
    for aln, set_of_ids in ALNs.items():
        yearly_aln_counts[year][aln] += len(set_of_ids)

## Restructure the data

The data is now structured incorrectly. That was an easy way to count, but not structured appropriately for output.

In [6]:
# I need ALN, then one value per year, as a row.

def get_aln_by_year(aln, year):
    return yearly_aln_counts[year].get(aln, 0)

def get_all_alns():
    all = set()
    for _, aln_counts in yearly_aln_counts.items():
        all.update(list(aln_counts.keys()))
    return all

unique_alns = sorted(list(get_all_alns()))

## Make a spreadsheet

Now, we'll take the data and produce a spreadsheet.

In [7]:
# Create a workbook
wb = openpyxl.Workbook()
# Grab the default sheet
sheet = wb.active

# Add the data.
# The header row should tell us what we're looking at
sheet.append(["Prefix", "Extension"] + config["YEARS"])

for aln in unique_alns:
    pfix = aln.split(".")[0]
    ext = aln.split(".")[1]
    values = []
    for year in config["YEARS"]:
        values.append(get_aln_by_year(aln, year))
    args = [pfix, ext] + values
    sheet.append(args)

wb.save("aln_report_counts_by_year.xlsx")

In [ ]:
# Metadata
for year, count in query_counts.items():
    print(f"Queries used in {year}: {count}")

Queries used in federal_awards: 177
